In [12]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random
import time
import string
import unicodedata
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
import multiprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import glob
import spacy.cli
spacy.cli.download("pt_core_news_sm")
import spacy
nlp = spacy.load("pt_core_news_sm")
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer
stemmer = RSLPStemmer()
from sklearn.metrics.pairwise import cosine_similarity
import ast
import numpy as np

# BASE_PATH = '/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/'
BASE_PATH = '/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold'

[nltk_data] Downloading package stopwords to /home/lucas-
[nltk_data]     nunes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lucas-
[nltk_data]     nunes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/lucas-
[nltk_data]     nunes/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package rslp to /home/lucas-nunes/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 65.3 MB/s  0:00:00 eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# 1. Carrega base consolidada

In [13]:
df_path = f'{BASE_PATH}/talent_pool_sample.parquet'
df = pd.read_parquet(df_path)
# df = df.head(100)

# 2. Métodos

In [14]:
def remove_person_names(text: str) -> str:
    """
    Remove nomes de pessoas do texto usando reconhecimento de entidades nomeadas (NER).
    
    Parâmetros:
        text (str): Texto de entrada.
    
    Retorna:
        str: Texto sem entidades de tipo "PER" (pessoas).
    """
    doc = nlp(text)
    return " ".join([token.text for token in doc if token.ent_type_ != "PER"])


def normalize_accents(text: str) -> str:
    """
    Remove acentos de um texto convertendo caracteres para ASCII.
    
    Parâmetros:
        text (str): Texto de entrada.
    
    Retorna:
        str: Texto sem acentos.
    """
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")


def remove_punctuation(text: str) -> str:
    """
    Remove pontuação de um texto, substituindo por espaço.
    
    Parâmetros:
        text (str): Texto de entrada.
    
    Retorna:
        str: Texto sem pontuação.
    """
    table = str.maketrans({key: " " for key in string.punctuation})
    return text.translate(table)


def normalize_str(text: str) -> str:
    """
    Normaliza texto aplicando:
      - Conversão para minúsculas
      - Remoção de números
      - Remoção de pontuação
      - Remoção de acentos
      - Normalização de espaços
    
    Parâmetros:
        text (str): Texto de entrada.
    
    Retorna:
        str: Texto normalizado.
    """
    text = text.lower()
    text = re.sub(r"\d+", " ", text)           # remove números
    text = remove_punctuation(text)            # remove pontuação
    text = normalize_accents(text)             # remove acentos
    text = re.sub(r"\s+", " ", text).strip()   # normaliza espaços
    return text


def tokenizer(text: str):
    """
    Tokeniza texto em português aplicando:
      - Normalização
      - Remoção de nomes de pessoas
      - Tokenização
      - Remoção de stopwords
      - Stemização
    
    Parâmetros:
        text (str): Texto de entrada.
    
    Retorna:
        list: Lista de tokens processados.
    """
    stop_words_br = set(nltk.corpus.stopwords.words("portuguese"))
    if isinstance(text, str):
        text = normalize_str(text)                                              # normaliza string
        text = remove_person_names(text)                                        # remove nomes de pessoas
        tokens = word_tokenize(text, language="portuguese")                     # tokeniza em português
        tokens = [t for t in tokens if t not in stop_words_br and len(t) > 2]   # remove stopwords e tokens curtos
        tokens = [stemmer.stem(t) for t in tokens]                              # aplica stemização
        return tokens
    return None


def tokenize_and_vectorize_fixed(df, campo_vetor, fitted_vectorizer, filename_prefix, batch_idx):
    """
    Transforma um lote de dados usando um vetorizador TF-IDF já treinado e salva o resultado como Parquet.
    
    Parâmetros:
        df (DataFrame): DataFrame contendo os dados de entrada.
        campo_vetor (str): Nome da coluna com o texto a ser vetorizado.
        fitted_vectorizer: Vetorizador TF-IDF já treinado.
        filename_prefix (str): Prefixo para nome dos arquivos de saída.
        batch_idx (int): Índice do lote.
    
    Retorna:
        DataFrame: DataFrame com os vetores TF-IDF.
    """
    # Transforma o texto em matriz TF-IDF usando vocabulário existente
    vector_matrix = fitted_vectorizer.transform(df[campo_vetor].fillna(""))

    # Cria DataFrame com os vetores
    df_tfidf = pd.DataFrame(
        vector_matrix.toarray(),
        columns=fitted_vectorizer.get_feature_names_out(),
        index=df.index
    )

    # Salva lote
    output_file = f"{filename_prefix}_batch_{batch_idx}.parquet"
    df_tfidf.to_parquet(output_file)
    print(f"Lote {batch_idx} salvo com formato {df_tfidf.shape} em {output_file}")

    return df_tfidf


def combine_vector_batches(batch_files, output_file):
    """
    Combina todos os arquivos de lotes em um único DataFrame e salva como Parquet.
    
    Parâmetros:
        batch_files (list): Lista de arquivos Parquet com vetores TF-IDF.
        output_file (str): Nome do arquivo final combinado.
    
    Retorna:
        DataFrame: DataFrame combinado com todos os vetores.
    """
    print("Combinando todos os lotes...")
    combined_dfs = []

    for i, file in enumerate(batch_files):
        df_batch = pd.read_parquet(file) 
        combined_dfs.append(df_batch)
        print(f"Lote {i} carregado: {df_batch.shape}")

    # Combina todos os DataFrames
    df_combined = pd.concat(combined_dfs, ignore_index=True)  # ignore_index=True reinicia os índices
    df_combined.to_parquet(output_file)
    print(f"Conjunto combinado salvo: {df_combined.shape} -> {output_file}")

    return df_combined


def combine_tfidf_batches(df, campo_vetor, vectorizer, batch_size=1000, output_dir="output"):
    """
    Treina o TF-IDF no dataset inteiro, processa em lotes e combina todos os lotes.
    
    Parâmetros:
        df (DataFrame): DataFrame de entrada.
        campo_vetor (str): Nome da coluna com os textos.
        vectorizer: Vetorizador TF-IDF.
        batch_size (int): Tamanho de cada lote (default=1000).
        output_dir (str): Diretório de saída (default="output").
    
    Retorna:
        tuple: (DataFrame combinado, vetorizador treinado, lista de arquivos de lotes).
    """
    # Treina o vetorizador em todos os dados
    print("Treinando o vetorizador em todo o conjunto de dados...")
    vectorizer.fit(df[campo_vetor].fillna(""))

    print(f"Tamanho do vocabulário: {len(vectorizer.vocabulary_)}")
    print("Exemplo de features:", list(vectorizer.get_feature_names_out())[:10])

    # Processa em lotes
    print("Processando lotes com vocabulário consistente...")
    filename_prefix = f"{BASE_PATH}/application_processed_{campo_vetor}"
    batch_files = []

    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        batch_idx = i // batch_size

        # Vetoriza lote
        #tokenize_and_vectorize_fixed(batch_df, campo_vetor, vectorizer, filename_prefix, batch_idx)
        X_tfidf, df_tfidf = vetoriza_input(df, campo_vetor, vectorizer)

        # Salva lote
        output_file = f"{filename_prefix}_batch_{batch_idx}.parquet"
        df_tfidf.to_parquet(output_file)
        print(f"Lote {batch_idx} salvo com formato {df_tfidf.shape} em {output_file}")

        batch_files.append(f"{filename_prefix}_batch_{batch_idx}.parquet")

        print(f"Lote {batch_idx} concluído (linhas {i} até {min(i+batch_size, len(df))})")

    print(f"\n{len(batch_files)} lotes processados com sucesso!")
    print(f"Todos os lotes agora possuem as mesmas {len(vectorizer.vocabulary_)} features")

    # Combina todos os lotes
    combined_output_file = f"{BASE_PATH}/talent_pool_vectors_combined_{campo_vetor}.parquet"
    df_tfidf_combined = combine_vector_batches(batch_files, combined_output_file)

    return df_tfidf_combined, vectorizer, batch_files


def vetoriza_input(df_input, campo_vetor, vectorizer):
    """
    Vetoriza um DataFrame usando TF-IDF e adiciona uma coluna com os vetores.
    
    Parâmetros:
        df_input (DataFrame): DataFrame de entrada.
        campo_vetor (str): Nome da coluna com os textos.
        vectorizer: Vetorizador TF-IDF.
    
    Retorna:
        tuple: (Matriz TF-IDF, DataFrame com coluna 'vetor_cv').
    """
    # Treina o vetorizador nos dados de texto
    vectorizer.fit(df_input[campo_vetor].fillna(""))

    # Transforma o texto em vetores TF-IDF
    X_tfidf = vectorizer.transform(df_input[campo_vetor].fillna(""))

    # Insere os vetores em uma coluna única no DataFrame
    df_tfidf = df_input.copy()
    df_tfidf['vetor_cv'] = [arr for arr in X_tfidf.toarray()]

    return X_tfidf, df_tfidf


def compute_similarity_batched(df_tfidf, campo_vetor, batch_size_sim=500, output_prefix='similarity_batch'):
    """
    Calcula a similaridade do cosseno em lotes para lidar com grandes conjuntos de dados.
    
    Parâmetros:
        df_tfidf (DataFrame ou matriz): Dados vetorizados.
        campo_vetor (str): Nome do campo usado para identificação.
        batch_size_sim (int): Tamanho do lote para cálculo da similaridade (default=500).
        output_prefix (str): Prefixo para os arquivos de saída (default='similarity_batch').
    
    Retorna:
        list: Lista de arquivos `.npy` contendo as matrizes de similaridade por lote.
    """
    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np

    n_samples = len(df_tfidf)
    print(f"Calculando similaridade para {n_samples} amostras em lotes de {batch_size_sim}")

    # Cria matriz de similaridade em lotes para gerenciar memória
    similarity_files = []

    for i in range(0, n_samples, batch_size_sim):
        batch_end = min(i + batch_size_sim, n_samples)
        batch_data = df_tfidf.iloc[i:batch_end]

        # Calcula similaridade entre este lote e TODOS os dados
        batch_similarity = cosine_similarity(batch_data, df_tfidf)

        # Salva similaridade do lote
        batch_file = f'{BASE_PATH}/{output_prefix}_{i}_{batch_end}_{campo_vetor}.npy'
        np.save(batch_file, batch_similarity)
        similarity_files.append(batch_file)

        print(f"Similaridade do lote {i}-{batch_end} calculada: {batch_similarity.shape}")

    return similarity_files

# 3. Vetorização e cálculo da similaridade

In [15]:
lista_campos_vetor = ['cv_pt'] # trocar para os campos a serem utilizados

for campo_vetor in lista_campos_vetor:
  print(f"Vetorizando campo: {campo_vetor}")
  print("Criando vocabulário a partir de todos os dados...")

  # Ajusta o vetorizador em TODOS os dados para criar vocabulário consistente
  vectorizer = TfidfVectorizer(
      tokenizer=tokenizer,
      max_features=10000,
      min_df=1,  # Allow terms that appear only once
      max_df=0.95  # Only remove terms that appear in 95% of documents
  )

  # Processa e combina batches
  df_talent_pool, vectorizer, batch_files = combine_tfidf_batches(df, campo_vetor, vectorizer, batch_size=100, output_dir="output")

Vetorizando campo: cv_pt
Criando vocabulário a partir de todos os dados...
Treinando o vetorizador em todo o conjunto de dados...


/home/lucas-nunes/workspace/Postech/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Tamanho do vocabulário: 7458
Exemplo de features: ['abaix', 'abap', 'abastec', 'abbott', 'abc', 'abend', 'abert', 'abi', 'abil', 'abilit']
Processando lotes com vocabulário consistente...
Lote 0 salvo com formato (300, 16) em /home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/application_processed_cv_pt_batch_0.parquet
Lote 0 concluído (linhas 0 até 100)


/home/lucas-nunes/workspace/Postech/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Lote 1 salvo com formato (300, 16) em /home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/application_processed_cv_pt_batch_1.parquet
Lote 1 concluído (linhas 100 até 200)


/home/lucas-nunes/workspace/Postech/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Lote 2 salvo com formato (300, 16) em /home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/application_processed_cv_pt_batch_2.parquet
Lote 2 concluído (linhas 200 até 300)

3 lotes processados com sucesso!
Todos os lotes agora possuem as mesmas 7458 features
Combinando todos os lotes...
Lote 0 carregado: (300, 16)
Lote 1 carregado: (300, 16)
Lote 2 carregado: (300, 16)
Conjunto combinado salvo: (900, 16) -> /home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/talent_pool_vectors_combined_cv_pt.parquet


# 4. Tratamento para input inicial de dados do Streamlit

In [16]:
# Usaremos o mesmo vetorizador que foi usado para os candidatos
campo_vetor = 'cv_candidato'

# DataFrame com os inputs do Streamlit
cv_input = [
'''
    assistente administrativo
santosbatista
itapecerica da serra/sp
29 anos ▪ brasileiro ▪ casado
formação acadêmica
 bacharel - ciências contábeis
centro universitário ítalo brasileiro
jul/2015 - dez/2018
 graduação - gestão financeira
centro universitário anhanguera
jan/2013 - dez/2014
habilidades
 contas a pagar e receber
 excel avançado
 indicadores kpi’s
 notas fiscais, cfop’s
 fechamento contábil
 emissão de boletos
 guias
 impostos
 budget
 controladoria
 sistemas integrados:
totvs;
folha matic;
navision
resumo profissional
profissional com experiência nos departamentos financeiro,
contábil, fiscal e controladoria jurídica. elaboração e análise de
indicadores kpi’s de resultado, relatórios, guias, gestão de
pagamentos, notas fiscais, boletos, fechamento financeiro e
contábil fiscal.
softwares erp protheus, folha matic, navision, elaw e sapiens,
excel avançado, (kpi's, painéis de dashboard e automatização).
histórico profissional
 01/2021 – 07/2021 fcn contabilidade freight forwarder

assistente contábil
conciliações contábeis, financeira, folha de pagamento,
fiscal, lançamentos contábeis, exportações txt, análise e
elaboração de relatórios, fechamento contábil, análise
fiscal e contabilização de folha de pagamento, sistema
folha matic.
 10/2020 – 01/2021 almeida advogados
assistente financeiro
gestão de pagamentos, baixa de boletos, relatórios gerenciais.
 04/2019 – 06/2019 fedex brasil logistica e transporte ltda
assistente juridico
responsável pelo fechamento mensal através das
apurações de provisões e reclassificações contábeis,
elaboração de indicadores financeiros e desempenho,
automatização de planilhas, análise de budget e real vs
orçado.
 07/2017 – 11/2018 atonanni construções e serviços ltda
assistente contábil / fiscal
lançamento de notas fiscais, apurações dos impostos (iss,
pis, cofins, cprb, ir, csll).
guias de pagamentos, sped fiscais, relatórios, xml, cfop,
ncm.
 06/2014 – 07/2017 iss servisytem do brasil ltda
assistente de controladoria
contas a pagar e a receber, análises contábeis e
financeiras, reembolsos, p.o’s.
gestão de custos, budget, real vs orçado, indicadores, kpi’s
e mapeamento de melhorias.
 04/2013 – 06/2014 n & n comércio de alimentos ltda
assistente financeiro
contas a pagar e a receber, boletos, relatórios gerenciais.
baixa de notas fiscais, concilação financeira, negociações
de pagamentos
'''
]
df_input = pd.DataFrame({'cv_candidato': cv_input})

# Transformar usando o mesmo vetorizador (não aplicar fit, apenas transform)
X_tfidf_input = vectorizer.transform(df_input[campo_vetor].fillna(""))

# Converter para formato de array e armazenar em um DataFrame
df_input_job_desc = pd.DataFrame({
    'vetor_cv': [X_tfidf_input.toarray()[0]]  # Armazena o vetor completo
})
df_input_job_desc = df_input_job_desc.reset_index(drop=True)

# 5. Sistema de Recomendação

In [17]:
class TalentRecommendationSystem:
    """
    Classe para recomendação de candidatos com base em similaridade de texto
    utilizando vetores TF-IDF e similaridade do cosseno.
    """

    def __init__(self, df_tfidf, df_tfidf_input, vectorizer):
        """
        Inicializa o sistema de recomendação.

        Parâmetros
        ----------
        df_tfidf : pd.DataFrame
            DataFrame contendo os vetores TF-IDF dos candidatos.
        df_tfidf_input : pd.DataFrame
            DataFrame contendo o vetor TF-IDF da descrição de vaga.
        vectorizer : TfidfVectorizer
            Vetorizador usado para transformar os textos.
        """
        self.df_tfidf = df_tfidf
        self.df_tfidf_input = df_tfidf_input
        self.vectorizer = vectorizer
        self.similarity_cache = {}

    def recommend_for_job_description(self, top_n=10):
        """
        Encontra os candidatos mais similares a uma descrição de vaga.

        Parâmetros
        ----------
        top_n : int, opcional, default=10
            Número de candidatos a retornar.

        Retorno
        -------
        results : list of dict
            Lista de dicionários com informações dos candidatos recomendados.
        """
        from sklearn.metrics.pairwise import cosine_similarity
        
        # Obtém o vetor da vaga e garante o formato correto
        job_vector = self.df_tfidf_input['vetor_cv'].values[0]
        if len(job_vector.shape) == 1:
            job_vector = job_vector.reshape(1, -1)
            
        # Obtém os vetores dos candidatos e garante o formato correto
        candidate_vectors = np.vstack([v for v in self.df_tfidf['vetor_cv'].values])
        
        # Garante que as dimensões sejam compatíveis
        if job_vector.shape[1] != candidate_vectors.shape[1]:
            raise ValueError(
                f"As dimensões dos vetores não coincidem: "
                f"vaga {job_vector.shape} vs candidatos {candidate_vectors.shape}"
            )
        
        # Calcula a similaridade com todos os candidatos
        similarities = cosine_similarity(job_vector, candidate_vectors)[0]

        # Seleciona os melhores candidatos
        top_indices = np.argsort(similarities)[::-1][:top_n]
        top_scores = similarities[top_indices]

        # Monta os resultados
        results = []
        for idx, score in zip(top_indices, top_scores):
            candidate_info = {
                'index': int(idx),
                'match_score': float(score),
                'nivel_profissional': self.df_tfidf.iloc[idx].get('nivel_profissional', 'N/A'),
                'area_atuacao': self.df_tfidf.iloc[idx].get('area_atuacao', 'N/A'),
                'nivel_academico': self.df_tfidf.iloc[idx].get('nivel_academico', 'N/A'),
                'conhecimentos_preview': str(self.df_tfidf.iloc[idx].get(campo_vetor, ''))[:200] + '...'
            }
            results.append(candidate_info)

        return results


# Inicializa o sistema de recomendação
print("Inicializando o Sistema de Recomendação de Talentos...")
talent_recommender = TalentRecommendationSystem(
    df_talent_pool,
    df_input_job_desc,
    vectorizer
)

print("Sistema de recomendação pronto!")
print(f"Foram carregados {len(df_talent_pool)} perfis de candidatos")
print(f"Tamanho do vocabulário: {len(vectorizer.vocabulary_)} atributos")

Inicializando o Sistema de Recomendação de Talentos...
Sistema de recomendação pronto!
Foram carregados 900 perfis de candidatos
Tamanho do vocabulário: 7458 atributos


In [18]:
matching_candidates = talent_recommender.recommend_for_job_description(top_n=10)

## Write

### Talent pool with vector

In [19]:
df_talent_pool.to_parquet(f'{BASE_PATH}/talent_pool_final.parquet')

### TF-IDF

### Alternative: Tokenizer-free vectorizer for easier deployment

In [ ]:
# Create a deployment-ready vectorizer without custom tokenizer
# This version uses standard preprocessing and is easier to load in production

# Pre-process the text data using our custom tokenizer
def preprocess_text_for_standard_vectorizer(text):
    """Apply our custom preprocessing but return as string for standard vectorizer"""
    if isinstance(text, str):
        tokens = tokenizer(text)  # Use our custom tokenizer
        return ' '.join(tokens) if tokens else ''
    return ''

# Apply preprocessing to the data
print("Preprocessing text data...")
df_preprocessed = df.copy()
df_preprocessed['cv_pt_processed'] = df_preprocessed['cv_pt'].apply(preprocess_text_for_standard_vectorizer)

# Create a standard vectorizer (no custom tokenizer)
standard_vectorizer = TfidfVectorizer(
    max_features=10000,
    min_df=1,
    max_df=0.95,
    lowercase=False,  # Already handled in preprocessing
    token_pattern=r'\S+',  # Split on whitespace only
    stop_words=None  # Already handled in preprocessing
)

# Train on preprocessed data
print("Training standard vectorizer...")
standard_vectorizer.fit(df_preprocessed['cv_pt_processed'].fillna(""))

# Save the standard vectorizer (easier to load)
standard_vectorizer_path = f'{BASE_PATH}/talent_vectorizer_standard.pkl'
joblib.dump(standard_vectorizer, standard_vectorizer_path)
print(f"✅ Standard vectorizer saved to: {standard_vectorizer_path}")

# Test loading
test_standard = joblib.load(standard_vectorizer_path)
print(f"✅ Standard vectorizer loaded successfully! Vocab size: {len(test_standard.vocabulary_)}")

# Also save the preprocessed data for consistency
df_preprocessed.to_parquet(f'{BASE_PATH}/talent_pool_preprocessed.parquet')
print("✅ Preprocessed data saved")

In [ ]:
# Save vectorizer
import joblib
import os

# Save to both locations for flexibility
vectorizer_paths = [
    f'{BASE_PATH}/talent_vectorizer.pkl',  # Data directory
    '/home/lucas-nunes/workspace/Postech/challenges/5_data/code/streamlit/talent_vectorizer.pkl'  # Streamlit directory
]

for path in vectorizer_paths:
    try:
        # Ensure directory exists
        os.makedirs(os.path.dirname(path), exist_ok=True)
        
        # Save vectorizer
        joblib.dump(vectorizer, path)
        print(f"✅ Vectorizer saved to: {path}")
        
        # Test loading immediately to verify it works
        test_load = joblib.load(path)
        print(f"✅ Vectorizer load test successful for: {path}")
        
    except Exception as e:
        print(f"❌ Error saving vectorizer to {path}: {e}")

print(f"\n📊 Vectorizer Info:")
print(f"   Vocabulary size: {len(vectorizer.vocabulary_)}")
print(f"   Max features: {vectorizer.max_features}")
print(f"   Tokenizer: {vectorizer.tokenizer.__name__ if hasattr(vectorizer.tokenizer, '__name__') else 'Custom function'}")


NameError: name 'vectorizer' is not defined

In [2]:
# Load vectorizer from .pkl (with proper tokenizer definition)
import joblib
import os

# First, ensure tokenizer function is available for unpickling
# The tokenizer function must be defined before loading the vectorizer
def tokenizer(text: str):
    """
    Tokeniza texto em português aplicando:
      - Normalização
      - Remoção de nomes de pessoas
      - Tokenização
      - Remoção de stopwords
      - Stemização
    
    Parâmetros:
        text (str): Texto de entrada.
    
    Retorna:
        list: Lista de tokens processados.
    """
    stop_words_br = set(nltk.corpus.stopwords.words("portuguese"))
    if isinstance(text, str):
        text = normalize_str(text)                                              # normaliza string
        text = remove_person_names(text)                                        # remove nomes de pessoas
        tokens = word_tokenize(text, language="portuguese")                     # tokeniza em português
        tokens = [t for t in tokens if t not in stop_words_br and len(t) > 2]   # remove stopwords e tokens curtos
        tokens = [stemmer.stem(t) for t in tokens]                              # aplica stemização
        return tokens
    return None

# Use relative path from BASE_PATH
vectorizer_path = f'{BASE_PATH}/talent_vectorizer.pkl'
print(f"Loading vectorizer from: {vectorizer_path}")

try:
    loaded_vectorizer = joblib.load(vectorizer_path)
    print("✅ Vectorizer loaded successfully!")
    print(f"Vocabulary size: {len(loaded_vectorizer.vocabulary_)}")
except FileNotFoundError:
    print(f"❌ Vectorizer file not found at: {vectorizer_path}")
    print("Please run the training cells first to generate the vectorizer.")
except Exception as e:
    print(f"❌ Error loading vectorizer: {e}")

# talent_recommender = TalentRecommendationSystem(
#     df_tfidf_combined,
#     df_tfidf_input,
#     loaded_vectorizer
# )

# matching_candidates = talent_recommender.recommend_for_job_description(top_n=10)

NameError: name 'BASE_PATH' is not defined

In [22]:
matching_candidates

[{'index': 635,
  'match_score': 0.3765331245693815,
  'nivel_profissional': 'Analista',
  'area_atuacao': 'N/A',
  'nivel_academico': 'Ensino Superior Completo',
  'conhecimentos_preview': '...'},
 {'index': 35,
  'match_score': 0.3765331245693815,
  'nivel_profissional': 'Analista',
  'area_atuacao': 'N/A',
  'nivel_academico': 'Ensino Superior Completo',
  'conhecimentos_preview': '...'},
 {'index': 335,
  'match_score': 0.3765331245693815,
  'nivel_profissional': 'Analista',
  'area_atuacao': 'N/A',
  'nivel_academico': 'Ensino Superior Completo',
  'conhecimentos_preview': '...'},
 {'index': 642,
  'match_score': 0.27760594374775704,
  'nivel_profissional': 'Analista',
  'area_atuacao': 'N/A',
  'nivel_academico': 'Pós Graduação Completo',
  'conhecimentos_preview': '...'},
 {'index': 42,
  'match_score': 0.27760594374775704,
  'nivel_profissional': 'Analista',
  'area_atuacao': 'N/A',
  'nivel_academico': 'Pós Graduação Completo',
  'conhecimentos_preview': '...'},
 {'index': 342